In [ ]:
# !python3 credito_featurizer.py --output-data-dir ./output_files --model-dir . --train ./input_files

In [1]:

import json
import joblib
import numpy as np
import pandas as pd
import os

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Binarizer, StandardScaler, OneHotEncoder ,OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.ensemble import RandomForestClassifier


def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [2]:
feature_columns_names = [
       'credito_coaplicante',
	   'credito_fiador',
       'financiamento_outro_lugar',
	   'tipo_emprego', 
	   'conta_corrente_valor',
       'outros_creditos_aqui',
	   'finance__credits__other_banks',
       'numero_dependentes',
	   'investimentos_valor',
	   'duracao_residencia',
       'valor_solicitado',
	   'tem_telefone', 
	   'duracao_emprego',
       'duracao_credito',
	   'historico_credito',
	   'financiamento_outros_bens',
       'proposito_credito',
	   'tipo_residencia',
	   'receita_disponivel']




feature_columns_dtype = {
       'credito_coaplicante'            : "float64",
	   'credito_fiador'                 : "float64", 
       'financiamento_outro_lugar'      : "float64",
	   'tipo_emprego'                   : "category" ,
	   'conta_corrente_valor'           : "category" ,
       'outros_creditos_aqui'           : "float64",
	   'finance__credits__other_banks'  : "float64",
       'numero_dependentes'             : "float64",
	   'investimentos_valor'            : "category" ,
	   'duracao_residencia'             : "category" ,
       'valor_solicitado'               : "float64",
	   'tem_telefone'                   : "category" ,
	   'duracao_emprego'                : "float64",
       'duracao_credito'                : "float64",
	   'historico_credito'              : "category" ,
	   'financiamento_outros_bens'      : "category" ,
       'proposito_credito'              : "category" ,
	   'tipo_residencia'                : "category" ,
	   'receita_disponivel'             : "float64"
}

label_column = 'deu_default'
label_column_dtype = { 'deu_default': "category"} 


In [4]:
##  arquivo de leitura, tem na primeira coluna o label 
raw_data = [ pd.read_csv(
    file, 
    header=None, 
    names=  feature_columns_names + [label_column] ,
    dtype=merge_two_dicts( feature_columns_dtype,label_column_dtype)) for file in ['/home/rubens/testes/input_files/credito_limpo.csv'] ]
concat_data = pd.concat(raw_data)

y= concat_data[label_column]


# Labels should not be preprocessed. predict_fn will reinsert the labels after featurizing.
concat_data.drop(label_column, axis=1, inplace=True)


In [5]:
category_features = ['tipo_emprego', 'conta_corrente_valor', 'investimentos_valor','duracao_residencia',
                     'tem_telefone', 'historico_credito', 'financiamento_outros_bens', 'proposito_credito',
                     'tipo_residencia', 'tipo_residencia'
                    ]

In [6]:
numeric_features = [ 'credito_coaplicante' ,'credito_fiador', 'financiamento_outro_lugar' , 'outros_creditos_aqui' ,           
                    'finance__credits__other_banks' , 'numero_dependentes' , 'valor_solicitado' ,               
                    'duracao_emprego' ,  'duracao_credito'  ]                


In [8]:
numeric_transformer = make_pipeline(
        SimpleImputer(strategy='median'),   ## replace NaN value
        StandardScaler())                   ## remove mean, and stddev =1

categorical_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value='missing'),    ## replace NaN value
        OneHotEncoder())

preprocessor = ColumnTransformer(transformers=[
             ("num", numeric_transformer, numeric_features),
             ("cat", categorical_transformer, category_features)
              ])

In [9]:
rfc_pipeline = Pipeline(steps = [
                     ('data_preprocessing', preprocessor),
                     ('model', RandomForestClassifier(max_depth = 10,
                        min_samples_leaf = 3,
                        min_samples_split = 4,
                        n_estimators = 200))
                ])

In [13]:
rfc_pipeline.named_steps['data_preprocessing'].transform(df_pred)

array([[-0.20676767, -0.23420572, -0.20926161,  1.02707891,  1.40836493,
        -0.42828957, -1.07086487, -0.73002164, -1.23647786,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ]])

In [10]:
# Fitting the training data to our pipeline
rfc_pipeline.fit(concat_data, y)
# Saving our pipeline to a binary pickle file
# joblib.dump(rfc_pipeline, 'model/rfc_pipeline.pkl')# Loading back in our serialized model

# loaded_model = joblib.load('model/rfc_pipeline.pkl')# Checking out our predicted results using the validation dataset

#pipeline_preds = rfc_pipeline.predict(concat_data)


Pipeline(memory=None,
         steps=[('data_preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                 

In [11]:

pipeline_preds = rfc_pipeline.predict(concat_data)
concat_data.head(1)

,credito_coaplicante,credito_fiador,financiamento_outro_lugar,tipo_emprego,conta_corrente_valor,outros_creditos_aqui,finance__credits__other_banks,numero_dependentes,investimentos_valor,duracao_residencia,valor_solicitado,tem_telefone,duracao_emprego,duracao_credito,historico_credito,financiamento_outros_bens,proposito_credito,tipo_residencia,receita_disponivel
0,0.0,0.0,0.0,operario,sem conta,2.0,1.0,1.0,muito_alta,2,1500.0,Nao,2.0,6.0,muito pobre,imovel,carro novo,proprio,2.0


In [12]:
entrada = [0,0,0,'operario','sem conta',2,1,1,'muito_alta',2,1500,'Nao',2,6,'muito pobre','imovel','carro novo','proprio',2]
df_pred = pd.DataFrame(np.array(entrada).reshape(-1,len(entrada)))
df_pred.columns = feature_columns_names
rfc_pipeline.predict(df_pred)

array(['0'], dtype=object)

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('t', preprocessor)])